# Домашнее задание 5 по курсу Байесовское мультимоделирование


Задача - продемонстрировать сходимость IGR(https://arxiv.org/pdf/1912.09588.pdf , оптмиизация KL-дивергенции) путём интерактивного графика на симплексе. Одно из распределений задано(q_prior), параметры второго оптимизируем(q_var).

IGR трансформирует нормальный шум $\varepsilon \sim N(0,I)$ в два шага:
$$y = \mu + \sigma \varepsilon$$
$$z = g(y, \tau)$$
g в последнем - некоторая обратимая функция,$\tau$ -- положительный гиперпараметр температуры. Пример такого преобразования, данный в статье выше и использованный в задании:
$$z_i = \frac{\exp(y_k/\tau)}{\sum_j \exp(y_j/\tau) + \delta }$$
$\delta$ -- дополнительный гиперпараметр,необходимый для обратимости преобразования. В задании он был принят за единицу. Ниже в коде подсчёт $z \rightarrow y$ происходит из этой формулы. Для посчета KL достаточно подсчитать KL для переменных y(см. статью)

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import math
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import tqdm
from tqdm.contrib import tzip
from scipy.stats import multivariate_normal
from scipy.stats import gamma
%matplotlib inline
#За исключением комменатрия далее это просто скопированный код для рисования распределения Дирихле внутри правильного 
#треугольника со стороной 1. (http://blog.bogatron.net/blog/2014/02/02/visualizing-dirichlet-distributions/)
#Для этого задания в нём потребовалось изменить одну функцию
corners = np.array([[0, 0], [1, 0], [0.5, 0.75**0.5]])
AREA = 0.5 * 1 * 0.75**0.5
triangle = tri.Triangulation(corners[:, 0], corners[:, 1])
refiner = tri.UniformTriRefiner(triangle)
trimesh = refiner.refine_triangulation(subdiv=4)
pairs = [corners[np.roll(range(3), -i)[1:]] for i in range(3)]
tri_area = lambda xy, pair: 0.5 * np.linalg.norm(np.cross(*(pair - xy)))

def xy2bc(xy, tol=1.e-4):
    '''Converts 2D Cartesian coordinates to barycentric.'''
    coords = np.array([tri_area(xy, p) for p in pairs]) / AREA
    return np.clip(coords, tol, 1.0 - tol)
#Ниже приведен тот самый изменённый класс, где считается правдоподобие нормального распределения, а не дирихле
class Multinomial(object):
    def __init__(self, mu, sigma, tau):
        self.mu = mu
        self.sigma = sigma
        self.tau = tau
    def pdf(self, x):
        x = x[0:2]/np.sum(x)#2 координаты заданы, третья считается по ним. Поэтому для подсчета правдоподобия необходимо 2
        #На сумму делим потому что треугольник не совсем правильный и даже 2 барицентрические координаты могут в сумме 
        #давать единицу. такой нормировки хватило для исправления ситуации
        sumx = (np.sum(x))/(1-np.sum(x))
        x = (x)*(sumx)#Для этой,следующей и предыдущей строчки - см формулу выше с описанием g. 
        #Ранее x - вектор z, после предыдущей строчки exp(y/tau)
        return multivariate_normal.pdf(np.log(x*self.tau) , mean = self.mu, cov = np.diag(self.sigma))
def draw_pdf_contours(mu,mu_2,sigma,sigma_2,tau,i, nlevels=200, subdiv=6, **kwargs):
    plt.subplot(1, 2, 1)
    plt.title("q_var")
    dist = Multinomial(mu[:,i],sigma[:,i],tau)
    refiner = tri.UniformTriRefiner(triangle)
    trimesh = refiner.refine_triangulation(subdiv=subdiv)
    pvals = [dist.pdf(xy2bc(xy)) for xy in zip(trimesh.x, trimesh.y)]
    plt.tricontourf(trimesh, pvals, nlevels, cmap='jet', **kwargs)
    plt.axis('equal')
    plt.xlim(0, 1)
    plt.ylim(0, 0.75**0.5)
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.title("q_prior")
    dist = Multinomial(mu_2,sigma_2,tau)
    refiner = tri.UniformTriRefiner(triangle)
    trimesh = refiner.refine_triangulation(subdiv=subdiv)
    pvals = [dist.pdf(xy2bc(xy)) for xy in zip(trimesh.x, trimesh.y)]
    plt.tricontourf(trimesh, pvals, nlevels, cmap='jet', **kwargs)
    plt.axis('equal')
    plt.xlim(0, 1)
    plt.ylim(0, 0.75**0.5)
    plt.axis('off')
    
    plt.show()
def draw_for_interaction(mu,mu_2,sigma,sigma_2,tau,i): 
    #Само по себе не несёт никакого смысла, просто не стал прописывать лишние fixed параметры в конце в interact
    draw_pdf_contours(mu,mu_2,sigma,sigma_2,tau,i)
    

Ячейка далее - градиенты KL-дивергенции двух нормальных распределений по параметрам первого.  

In [2]:
size = 2#другое значение параметра кажется неестественным в задании и с его изменением код не будет работать
        #Но удалять не стал - это просто размерность генерируемых векторов
def grad_mu(mu_1,mu_2,sigma_1,sigma_2,size = size):    
    return 2*(mu_1 - mu_2)/(sigma_2)
def grad_sigma(mu_1,mu_2,sigma_1,sigma_2):
    return np.ones(size)/(sigma_2) - np.ones(size)/(sigma_1)

In [3]:
def gradient_descent(mu_1,mu_2,sigma_1,sigma_2,size = size):
    muarr = mu_1.reshape(size,1)
    sigmaarr = sigma_1.reshape(size,1)
    i = 0
    while (np.linalg.norm(mu_1 - mu_2) + np.linalg.norm(sigma_1 - sigma_2) > 0.1) and (i < 1000):
        i += 1
        mu_1 = mu_1 - grad_mu(mu_1,mu_2,sigma_1,sigma_2)
        sigma_1 = sigma_1 - 3*grad_sigma(mu_1,mu_2,sigma_1,sigma_2)
        muarr = np.append(muarr,mu_1.reshape(size,1),axis = 1)
        sigmaarr = np.append(sigmaarr,sigma_1.reshape(size,1),axis = 1)
    return(muarr,sigmaarr)
def get_visualisation_params(): #случайная генерация параметров эксперимента
    size = 2
    mu_1 = np.random.uniform(-10, 10, size )
    sigma_1 = np.random.uniform(1, 10, size ) #начальное q_var
    mu_2 = np.random.uniform(-10, 10, size ) 
    sigma_2 = np.random.uniform(1, 10, size ) #q_prior
    tau = gamma.rvs(1)
    muarr,sigmaarr = gradient_descent(mu_1,mu_2,sigma_1,sigma_2)
    return(mu_2,sigma_2,muarr,sigmaarr,tau)


Предупреждение: следующий интеракт обновляется за несколько секунд, т е не сразу. Для ускорения можно уменьшить параметр subdiv
функции draw_pdf_contours

In [4]:
mu_2,sigma_2,muarr,sigmaarr,tau = get_visualisation_params()
interact(draw_for_interaction,i = widgets.IntSlider(value = 1,
                                               min = 0,
                                               max = len(muarr[0]),
                                               step = 1) , mu = fixed(muarr), mu_2 = fixed(mu_2),
                                                           sigma = fixed(sigmaarr),  sigma_2 = fixed(sigma_2),tau = fixed(tau))

interactive(children=(IntSlider(value=1, description='i', max=24), Output()), _dom_classes=('widget-interact',…

<function __main__.draw_for_interaction(mu, mu_2, sigma, sigma_2, tau, i)>

Комментарий о результатах в конце можно описать кратко: нужно подбирать гиперпараметры. Например, я генерирую матожидание и дисперсию из равномерного распределения, скорее всего, было бы лучше из нормального/гамма. Выбор основан на том, что в большом матожидании/дисперсии нет смысла и это просто дает синие трегуольники на графике с едва(а то и совсем не) видимыми красными фрагментами. Наивная реализация градиентного спуска сработала, более того, весь процесс как правило виден к ~20 шагу, то есть можно было ослабить критерий остановки. 